In [61]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
import json
import numpy as np

from api_keys import g_key

In [62]:
#Create a list of Cities and States to pull cooridinates from Google Place Search
City_df = pd.DataFrame(
    {"City":['Lockhart','Memphis', "Kansas%City", 'Lexington','Austin', 'St.%Louis', 'Dallas', 'Nashville', 'Atlanta',
              'Brooklyn'],
     "State":['TX', 'TN', 'MS', 'NC', 'TX', 'MS', 'TX', 'TN', 'GA', 'NY']
    }
) 

City_df.head(10)

,City,State
0,Lockhart,TX
1,Memphis,TN
2,Kansas%City,MS
3,Lexington,NC
4,Austin,TX
5,St.%Louis,MS
6,Dallas,TX
7,Nashville,TN
8,Atlanta,GA
9,Brooklyn,NY


In [63]:
base_url = "https://maps.googleapis.com/maps/api/geocode/json?"

Cities = ['{}, {}'.format(City, State) for City, State in zip(City_df['City'], City_df['State'])]
p_requests = [requests.get(base_url, params={
    'address':City,
    'key':g_key
}) for City in Cities]



#print(p_requests)

places_data = [place.json() for place in p_requests]
places_data


[{'results': [{'address_components': [{'long_name': 'Lockhart',
      'short_name': 'Lockhart',
      'types': ['locality', 'political']},
     {'long_name': 'Caldwell County',
      'short_name': 'Caldwell County',
      'types': ['administrative_area_level_2', 'political']},
     {'long_name': 'Texas',
      'short_name': 'TX',
      'types': ['administrative_area_level_1', 'political']},
     {'long_name': 'United States',
      'short_name': 'US',
      'types': ['country', 'political']},
     {'long_name': '78644', 'short_name': '78644', 'types': ['postal_code']}],
    'formatted_address': 'Lockhart, TX 78644, USA',
    'geometry': {'bounds': {'northeast': {'lat': 29.9244709,
       'lng': -97.62464390000001},
      'southwest': {'lat': 29.840644, 'lng': -97.72831509999999}},
     'location': {'lat': 29.8849441, 'lng': -97.6699996},
     'location_type': 'APPROXIMATE',
     'viewport': {'northeast': {'lat': 29.9244709, 'lng': -97.62464390000001},
      'southwest': {'lat': 29.8406

In [64]:
[place['results'][0] for place in places_data if len(place['results']) > 0]

#city_list = [h_attribute['results'][0]['address_components']['long_name'] for h_attribute in places_data if len(h_attribute['results']) > 0]
lat_list = [h_attribute['results'][0]['geometry']['location']['lat'] for h_attribute in places_data if len(h_attribute['results']) > 0]
lng_list = [h_attribute['results'][0]['geometry']['location']['lng'] for h_attribute in places_data if len(h_attribute['results']) > 0]

coordinates_df = pd.DataFrame(data={'Lat': lat_list, 'Lng': lng_list }, copy=True)
coordinates_df.head(10)






,Lat,Lng
0,29.884944,-97.670000
1,35.149534,-90.048980
2,39.099727,-94.578567
3,35.824027,-80.253384
4,30.267153,-97.743061
5,30.308808,-89.330046
6,32.776664,-96.796988
7,36.162664,-86.781602
8,33.748995,-84.387982
9,40.678178,-73.944158


In [65]:
# Merge the SFact_DF and line_DF into a combined dataframe
coordinates_df = pd.merge(City_df, coordinates_df, left_index = True, right_index = True)
coordinates_df.head(11)

,City,State,Lat,Lng
0,Lockhart,TX,29.884944,-97.670000
1,Memphis,TN,35.149534,-90.048980
2,Kansas%City,MS,39.099727,-94.578567
3,Lexington,NC,35.824027,-80.253384
4,Austin,TX,30.267153,-97.743061
5,St.%Louis,MS,30.308808,-89.330046
6,Dallas,TX,32.776664,-96.796988
7,Nashville,TN,36.162664,-86.781602
8,Atlanta,GA,33.748995,-84.387982
9,Brooklyn,NY,40.678178,-73.944158


In [68]:
#Remove % from Kansas%City and St.%Louis
coordinates_df['City'] = coordinates_df['City'].replace(
    {'Kansas%City': 'Kansas City', 'St.%Louis' : 'St. Louis'})
coordinates_df.head(11)

,City,State,Lat,Lng
0,Lockhart,TX,29.884944,-97.670000
1,Memphis,TN,35.149534,-90.048980
2,Kansas City,MS,39.099727,-94.578567
3,Lexington,NC,35.824027,-80.253384
4,Austin,TX,30.267153,-97.743061
5,St. Louis,MS,30.308808,-89.330046
6,Dallas,TX,32.776664,-96.796988
7,Nashville,TN,36.162664,-86.781602
8,Atlanta,GA,33.748995,-84.387982
9,Brooklyn,NY,40.678178,-73.944158


In [69]:
#Export to file as CSV
coordinates_df.to_csv("Output/Coordinates.csv", index=True, header = True)